<a href="https://colab.research.google.com/github/playeredlc/treinamento-h2ia/blob/master/Buscas/buscas_sem_informa%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# O Problema
Sliding Puzzle - Bloco Deslizante

In [1]:
# !wget -qq https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif
from IPython.display import Image
Image(url='https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif',width=200)

# Resolver o quebra-cabeças usando Buscas

In [2]:
import numpy as np
from collections import deque

## Definição do nodo

Cada nodo é um objeto que possui:
* Estado, ou seja, a configuração das peças. 
* Nodo pai, aquele nodo a partir de qual ele foi gerado.
* Caminho até a raiz, uma lista de nodos que refaz o caminho de volta até a raiz.

Cada nodo implementa os métodos que permitem movimentar a peça vazia (quando permitido) gerando novos estados. O método para gerar todos os filhos possíveis a partir do seu estado atual e o método que faz o backtracking até a raiz.

In [3]:
class Node:
  def __init__(self, state, parent=None):
    self.state = state
    self.parent = parent
    self.path_to_root = []
  
  def get_zero(self):
    coord = np.where(self.state == 0)
    r, c = coord[0][0], coord[1][0]

    return r, c

  def move_left(self):
    r, c = self.get_zero()
    if(c != 0):
      new_st = self.state.copy()
      # swap
      new_st[r, c], new_st[r, c-1] = new_st[r, c-1], new_st[r, c]
      return new_st    
    else:
      return np.empty(0)
  
  def move_up(self):
    r, c = self.get_zero()
    if(r != 0):
      new_st = self.state.copy()
      # swap
      new_st[r, c], new_st[r-1, c] = new_st[r-1, c], new_st[r, c]
      return new_st    
    else:
      return np.empty(0)
  
  def move_right(self):
    r, c = self.get_zero()
    if(c < self.state.shape[1]-1):
      new_st = self.state.copy()
      # swap
      new_st[r, c], new_st[r, c+1] = new_st[r, c+1], new_st[r, c]
      return new_st    
    else:
      return np.empty(0)

  def move_down(self):
    r, c = self.get_zero()
    if(r < self.state.shape[0]-1):
      new_st = self.state.copy()
      # swap
      new_st[r, c], new_st[r+1, c] = new_st[r+1, c], new_st[r, c]
      return new_st
    else:
      return np.empty(0)

  def generate_children(self):
    children_list = []

    child_state = self.move_left()
    if(child_state.any()):
      children_list.append(Node(child_state, self))
    
    child_state = self.move_up()
    if(child_state.any()):
      children_list.append(Node(child_state, self))
    
    child_state = self.move_right()
    if(child_state.any()):
      children_list.append(Node(child_state, self))
    
    child_state = self.move_down()
    if(child_state.any()):
      children_list.append(Node(child_state, self))

    return children_list

  def generate_path(self):
    self.path_to_root.clear()
    previous = self.parent
    while(previous.parent != None):
      self.path_to_root.append(previous.state.copy())
      previous = previous.parent
    
    return self.path_to_root

  def print_solution(self):
    move = 1
    for state in reversed(self.path_to_root):
      print(f'\nMove {move}:\n{state}')
      move += 1
    print(f'\nFinal move:\n{self.state}')
  

## Busca em largura

Para implementação da busca em largura utiliza-se uma estrutura de fila para armazenar os nodos que devem ser explorados, esse tipo de estrutura garante que todos os nodos de um mesmo nível sejam explorados antes de passar para o próximo nível.

Utiliza-se também um set para guardar os estados que foram gerados e já foram desempilhados, a fim de evitar a repetição de estados.

Em questão de completude a busca em largura é completa, ou seja, sempre encontra a solução caso ela existe. Quando encontrada, essa solução é ótima.

In [4]:
# https://docs.python.org/3/library/collections.html#collections.deque

final_state = np.array([[1, 2, 3],
                        [4, 5, 6],
                        [7, 8, 0]])

def breadth_first_search(initial_state, final_state):
  it = 0
  q = deque()
  visited_states = set()
  
  root = Node(initial_state)
  q.insert(0, root)

  while(not q.count(0)):
        
    print(f'\nIteration {it}...')
    print(f'Visited states amount: {len(visited_states)}')
    
    node = q.pop()

    if(np.array_equal(node.state, final_state)):
      print(f'\nRESULT FOUND!\n{node.state}')
      node.generate_path()
      return node
    
    children_list = node.generate_children()
    
    for child in children_list:
      if((not (np.array2string(child.state)) in visited_states)):
        q.insert(0, child)

    visited_states.add(np.array2string(node.state))
    it += 1

  return False

### Teste para uma instância simples do problema

Resolvido em 14 iterações, gerando uma solução com 3 movimentos (solução ótima).

In [5]:
%%time

initial_state = np.array([[1, 2, 3],
                          [4, 6, 0],
                          [7, 5, 8]])

res = breadth_first_search(initial_state, final_state)


Iteration 0...
Visited states amount: 0

Iteration 1...
Visited states amount: 1

Iteration 2...
Visited states amount: 2

Iteration 3...
Visited states amount: 3

Iteration 4...
Visited states amount: 4

Iteration 5...
Visited states amount: 5

Iteration 6...
Visited states amount: 6

Iteration 7...
Visited states amount: 7

Iteration 8...
Visited states amount: 8

Iteration 9...
Visited states amount: 9

Iteration 10...
Visited states amount: 10

Iteration 11...
Visited states amount: 11

Iteration 12...
Visited states amount: 12

Iteration 13...
Visited states amount: 13

Iteration 14...
Visited states amount: 14

RESULT FOUND!
[[1 2 3]
 [4 5 6]
 [7 8 0]]
CPU times: user 21.1 ms, sys: 0 ns, total: 21.1 ms
Wall time: 28.2 ms


In [6]:
print(f'Initial state:\n{initial_state}')
res.print_solution()

Initial state:
[[1 2 3]
 [4 6 0]
 [7 5 8]]

Move 1:
[[1 2 3]
 [4 0 6]
 [7 5 8]]

Move 2:
[[1 2 3]
 [4 5 6]
 [7 0 8]]

Final move:
[[1 2 3]
 [4 5 6]
 [7 8 0]]


## Busca em Profundidade

A busca em profundidade é implementada de maneira semelhante a busca em largura, com a diferença fundamental de que se utiliza uma estrutura de pilha para armazenar os nodos que deverão ser explorados na sequência. A diferença nas estruturas faz com que um ramo seja expandido até que não seja mais possível ou a solução encontrada.

Quando os nodos visitados são controlados para evitar loops, a busca em profundidade, assim como a busca em largura é completa, porém, a solução encontrada não é garantidamente ótima.

In [7]:
final_state = np.array([[1, 2, 3],
                        [4, 5, 6],
                        [7, 8, 0]])

def depth_first_search(initial_state, final_state):
  it = 0
  q = deque()
  visited_states = set()
  
  root = Node(initial_state)
  q.insert(0, root)

  while(not q.count(0)):
        
    print(f'\nIteration {it}...')
    print(f'Visited states amount: {len(visited_states)}')
    
    node = q.pop()

    if(np.array_equal(node.state, final_state)):
      print(f'\nRESULT FOUND!\n{node.state}')
      node.generate_path()
      return node
    
    children_list = node.generate_children()
    
    for child in children_list:
      if((not (np.array2string(child.state)) in visited_states)):
        q.append(child)

    visited_states.add(np.array2string(node.state))
    it += 1

  return False

### Teste para uma instância simples do problema

A partir do mesmo estado inicial usado no teste com a busca em largura, obteve-se uma solução em 9 iterações (menor que na busca em largura), porém, essa solução requer 9 movimentos para ser encontrada (solução ótima possui 3).

In [8]:
%%time

initial_state = np.array([[1, 2, 3],
                          [4, 6, 0],
                          [7, 5, 8]])

res = depth_first_search(initial_state, final_state)


Iteration 0...
Visited states amount: 0

Iteration 1...
Visited states amount: 1

Iteration 2...
Visited states amount: 2

Iteration 3...
Visited states amount: 3

Iteration 4...
Visited states amount: 4

Iteration 5...
Visited states amount: 5

Iteration 6...
Visited states amount: 6

Iteration 7...
Visited states amount: 7

Iteration 8...
Visited states amount: 8

Iteration 9...
Visited states amount: 9

RESULT FOUND!
[[1 2 3]
 [4 5 6]
 [7 8 0]]
CPU times: user 14.2 ms, sys: 1.03 ms, total: 15.2 ms
Wall time: 17.5 ms


In [9]:
print(f'Initial state:\n{initial_state}')
res.print_solution()

Initial state:
[[1 2 3]
 [4 6 0]
 [7 5 8]]

Move 1:
[[1 2 3]
 [4 6 8]
 [7 5 0]]

Move 2:
[[1 2 3]
 [4 6 8]
 [7 0 5]]

Move 3:
[[1 2 3]
 [4 0 8]
 [7 6 5]]

Move 4:
[[1 2 3]
 [4 8 0]
 [7 6 5]]

Move 5:
[[1 2 3]
 [4 8 5]
 [7 6 0]]

Move 6:
[[1 2 3]
 [4 8 5]
 [7 0 6]]

Move 7:
[[1 2 3]
 [4 0 5]
 [7 8 6]]

Move 8:
[[1 2 3]
 [4 5 0]
 [7 8 6]]

Final move:
[[1 2 3]
 [4 5 6]
 [7 8 0]]


## Teste com uma instância difícil do problema

Os dois algoritmos foram utilizados para resolver a [instância mais difícil](http://w01fe.com/blog/2009/01/the-hardest-eight-puzzle-instances-take-31-moves-to-solve/) pra esse problema. A partir desse estado inicial são necessários no minimo 31 movimentos para chegar a solução.

O número máximo de estados únicos que podem ser gerados para esse problema é `9!/2 = 181440`.

Na busca em largura todos os estados foram gerados em 519163 iterações, e como esperado a solução ótima foi encontrada com 31 movimentos.Esse processo levou aproximandamente 28 minutos.

`CPU times: user 26min 20s, sys: 1min 45s, total: 28min 6s
Wall time: 26min 30s`

Na busca em profundidade a execução foi muito mais rápida, necessitando 58242 iterações para encontrar a solução e executando em aproximadamente 2 minutos. Entretanto, são necessários 57299 movimentos para chegar do estado inicial a solução utilizando o caminho encontrado.

`CPU times: user 1min 49s, sys: 11.9 s, total: 2min 1s
Wall time: 1min 50s`

Certamente é possível otimizar a implementação dos nodos e os algoritmos das buscas para melhorar o desempenho, porém, mesmo com a implementação atual é possível ter uma boa base para comparação entre as diferentes estratégias.

### BFS

In [10]:
%%time

initial_state = np.array([[8, 6, 7],
                          [2, 5, 4],
                          [3, 0, 1]])

res = breadth_first_search(initial_state, final_state)

A saída de streaming foi truncada nas últimas 5000 linhas.
Visited states amount: 181438

Iteration 517500...
Visited states amount: 181438

Iteration 517501...
Visited states amount: 181438

Iteration 517502...
Visited states amount: 181438

Iteration 517503...
Visited states amount: 181438

Iteration 517504...
Visited states amount: 181438

Iteration 517505...
Visited states amount: 181438

Iteration 517506...
Visited states amount: 181438

Iteration 517507...
Visited states amount: 181438

Iteration 517508...
Visited states amount: 181438

Iteration 517509...
Visited states amount: 181438

Iteration 517510...
Visited states amount: 181438

Iteration 517511...
Visited states amount: 181438

Iteration 517512...
Visited states amount: 181438

Iteration 517513...
Visited states amount: 181438

Iteration 517514...
Visited states amount: 181438

Iteration 517515...
Visited states amount: 181438

Iteration 517516...
Visited states amount: 181438

Iteration 517517...
Visited states amount: 

In [11]:
print(f'Número de movimentos no caminho encontrado (bfs): {len(res.path_to_root)+1}')
# res.print_solution()

Número de movimentos no caminho encontrado (bfs): 31


### DFS

In [12]:
%%time

initial_state = np.array([[8, 6, 7],
                          [2, 5, 4],
                          [3, 0, 1]])

res = depth_first_search(initial_state, final_state)

A saída de streaming foi truncada nas últimas 5000 linhas.
Visited states amount: 56578

Iteration 56579...
Visited states amount: 56579

Iteration 56580...
Visited states amount: 56580

Iteration 56581...
Visited states amount: 56581

Iteration 56582...
Visited states amount: 56582

Iteration 56583...
Visited states amount: 56583

Iteration 56584...
Visited states amount: 56584

Iteration 56585...
Visited states amount: 56585

Iteration 56586...
Visited states amount: 56586

Iteration 56587...
Visited states amount: 56587

Iteration 56588...
Visited states amount: 56588

Iteration 56589...
Visited states amount: 56589

Iteration 56590...
Visited states amount: 56590

Iteration 56591...
Visited states amount: 56591

Iteration 56592...
Visited states amount: 56592

Iteration 56593...
Visited states amount: 56593

Iteration 56594...
Visited states amount: 56594

Iteration 56595...
Visited states amount: 56595

Iteration 56596...
Visited states amount: 56596

Iteration 56597...
Visited st

In [13]:
print(f'Número de movimentos no caminho encontrado (dfs): {len(res.path_to_root)+1}')
# res.print_solution()

Número de movimentos no caminho encontrado (dfs): 57299


## Discorra sobre o desempenho dos métodos em questões de:


1.   Consumo de memória
2.   Processamento

A complexidade espacial da busca em largura, no geral, costuma ser maior que a da busca em profundidade. Isso porque a busca em largura expande todos os nodos de um determinado nível antes de prosseguir para o próximo. Dependendo da profundidade em que a solução se encontra e o fator de ramificação da árvore, a quantidade de memória necessária para armazenar esses nodos pode crescer muito rapidamente. Entretanto, dependendo da instância do problema a busca em largura pode encontrar a solução utilizando menos memória que a busca em profundidade.

Com relação ao tempo de processamento, no geral a busca em profundidade leva vantagem, como observado no teste realizado para a instância mais difícil do problema, em que a diferença no tempo para encontrar uma solução válida foi muito grande (~2min vs ~28min). É importante relembrar que apesar de ser mais rápido, a solução encontrada pela busca em profundidade não é ótima e no geral é muito mais custosa.